# Purpose

This notebook is to test implementation of Ensemble, Flipout and DUQ models.
- Question: Can the ensembles use dropout and dropconnect weights? Or are they to be trained without either.

In [2]:
# Model for ensemble:
# model = DeepEnsembleClassifier(lambda: build_standard_model(dropout_best_hps), num_estimators=10)     100 epochs
# DUQ: 200 epochs
# Flipout: 200 epochs

In [1]:
%env CUDA_VISIBLE_DEVICES=1

env: CUDA_VISIBLE_DEVICES=1


In [1]:
import sys, os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
os.environ["CUDA_VISIBLE_DEVICES"]="1" 
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from models_bachelors import *
from file_functions import *
import tensorflow as tf
import keras_tuner as kt
from keras_uncertainty.models import StochasticClassifier, DeepEnsembleClassifier
import time

Using TensorFlow backend


Keras Uncertainty will use standalone Keras backend

In [2]:
'''
Load best hyperparams
'''

n_epochs= 200
early_stopping = EarlyStopping(monitor='val_loss', patience=10)
subject_ids = [0, 1, 2, 3, 4, 5, 6, 7, 8]
'''
Load data
'''
dataset = load('all_subjects_runs_no_bandpass')
lockbox = load('lockbox')['data']
loaded_inputs = dataset['inputs']
loaded_targets = dataset['targets']

In [3]:
methods = {'flipout': [], 'duq': [], 'standard_dropout': [], 'ensemble_dropout': [], 
           'standard_dropconnect': [], 'mcdropconnect': [], 'mcdropout': []}
x_train_shape_0 = None

In [10]:
# x_train_shape param required for flipout
def create_model(method):
    if method == 'flipout':
        return build_flipout_model(load_tuned_flipout())
    elif method == 'duq':
        return build_duq_model(load_tuned_duq())
    elif 'dropout' in method or 'dropconnect' in method:
        dropout_hp, dropconnect_hp = load_tuned_models()
        if method == 'standard_dropout':
            return build_standard_model(dropout_hp)
        elif method == 'standard_dropconnect':
            return build_standard_model_dropconnect(dropconnect_hp)
        elif method == 'mcdropout':
            return build_dropout_model(dropout_hp)
        elif method == 'mcdropconnect':
            return build_dropconnect_model(dropconnect_hp)
        elif method == 'ensemble_dropout':
            return build_ensemble_model()      

In [6]:
'''
Training Loop
'''
for method, times_lst in methods.items():
    for test_subject_id in subject_ids:
        train_ids = subject_ids[:]
        train_ids.remove(test_subject_id)       # Remove test subject id
        test_subj_lockbox = lockbox[test_subject_id]        # Get lockbox indexes (8, 57) for the test subject
        inputs = loaded_inputs[train_ids]           # Get train set inputs
        targets = loaded_targets[train_ids]         # Get train set targets
        inputs, targets = remove_lockbox(inputs, targets, test_subj_lockbox)    # Remove lockboxed set from train set
        X_train, X_val, Y_train, Y_val = train_test_split(inputs, targets,test_size=0.1) 
        model = create_model(method, X_train.shape[0])
        start_time = time.time()
        history = model.fit(X_train, Y_train, epochs=n_epochs, validation_data=[X_val, Y_val],
                        callbacks=[early_stopping], verbose=0)
        end_time = time.time() - start_time
        times_lst.append(end_time)
        

Reloading Tuner from flipout/tuning/flipout_2/tuner0.json


/home/pmanivannan/miniconda3/envs/tf/lib/python3.9/site-packages/tensorflow/python/util/dispatch.py:1260: UserWarning: `tf.keras.backend.random_binomial` is deprecated, and will be removed in a future version.Please use `tf.keras.backend.random_bernoulli` instead.
  return dispatch_target(*args, **kwargs)


KeyboardInterrupt: 

In [8]:
for method, lsts in methods.items():
    print(f'\n{method}: {np.mean(lsts)} +/- {np.std(lsts)}\n')


flipout: 147.4766283300188 +/- 23.39263702058503


duq: 106.92008466190762 +/- 20.217598694642668


standard_dropout: 104.87643946541681 +/- 18.1966711701486


ensemble_dropout: 1123.7308384312523 +/- 102.18134506083021


standard_dropconnect: 129.40478091769748 +/- 32.127607006211235


mcdropconnect: 111.53497536977132 +/- 20.427049771346518


mcdropout: 105.2778045601315 +/- 19.51180702084185



flipout: 147.4766283300188 +/- 23.39263702058503


duq: 106.92008466190762 +/- 20.217598694642668


standard_dropout: 104.87643946541681 +/- 18.1966711701486


ensemble_dropout: 1123.7308384312523 +/- 102.18134506083021


standard_dropconnect: 129.40478091769748 +/- 32.127607006211235


mcdropconnect: 111.53497536977132 +/- 20.427049771346518


mcdropout: 105.2778045601315 +/- 19.51180702084185

In [11]:
prediction_times = {'standard_dropout': [], 'flipout': [], 'duq': [], 'ensemble_dropout': [], 
           'standard_dropconnect': [], 'mcdropconnect': [], 'mcdropout': []}

def get_weights_directory(method):
    if method in ['standard', 'standard_dropout', 'mcdropout']:
        return f'mcdropout/weights'
    elif method in ['dropconnect', 'standard_dropconnect', 'mcdropconnect']:
        return f'mcdropconnect/weights'
    else:
        return f'{method}/weights'
    
# checkpoint path is sometimes an hdf5 file in the case of ensembles and a ckpt file for mcdropout and the like.
# this function corrects the weight path depending on the extension
# duq, ensembles and flipout don't have .ckpt but mcdropout and mcdropconnect do
def rectify_wts_path(method, wts_path):
    if method in ['standard', 'standard_dropout', 'standard_dropconnect', 'mcdropout', 'mcdropconnect']:
        return wts_path + '.ckpt'
    else:
        return wts_path
    
def get_predictions(method, X_test, X_lock, wts_path):
    wts_path = rectify_wts_path(method, wts_path)
    if method in ['mcdropconnect', 'mcdropout', 'flipout']:
        model = create_model(method)
        model.load_weights(wts_path).expect_partial()
        model = StochasticClassifier(model)
        start_time = time.time()
        Y_preds = model.predict_samples(X_test, num_samples=50, batch_size=1)
        prediction_times[method].append(time.time() - start_time)    # global variable in this case is bad practice ik, but it's quick
        lockbox_Y_preds = model.predict_samples(X_lock, num_samples=50, batch_size=1)
    else:
        model = create_model(method)
        if 'ensemble' in method:
            model.load_weights(wts_path)
        else:
            model.load_weights(wts_path).expect_partial()
        start_time = time.time()
        Y_preds = model.predict(X_test, batch_size=1)
        prediction_times[method].append(time.time() - start_time)
        # Get lockboxed Y_preds for test subject
        lockbox_Y_preds = model.predict(X_lock, batch_size=1)
    
    return Y_preds, lockbox_Y_preds

In [12]:
for method, lsts in prediction_times.items():
    print(f'{method}')
    wts_directory = get_weights_directory(method)
    # Iterate through test subjects
    for test_subject_id in range(0, 9):
        print(f'test subject {test_subject_id}')
        train_subj_ids = [x for x in subject_ids if x != test_subject_id]
        X_test = loaded_inputs[test_subject_id]
        Y_true = loaded_targets[test_subject_id]
        X_lock, Y_lock = get_lockbox_data(loaded_inputs[train_subj_ids], loaded_targets[train_subj_ids], lockbox[test_subject_id])
        wts_path = f'{wts_directory}/test_subj_{test_subject_id}'
        # Get Y_preds for test subject
        Y_preds, lockbox_Y_preds = get_predictions(method, X_test, X_lock, wts_path)

standard_dropout
test subject 0
Reloading Tuner from mcdropout/tuning/mcdropout/tuner0.json
Reloading Tuner from mcdropconnect/tuning/mcdropconnect/tuner0.json


456/456 [==============================] - 0s 859us/step
test subject 1
Reloading Tuner from mcdropout/tuning/mcdropout/tuner0.json
Reloading Tuner from mcdropconnect/tuning/mcdropconnect/tuner0.json
456/456 [==============================] - 0s 865us/step
test subject 2
Reloading Tuner from mcdropout/tuning/mcdropout/tuner0.json
Reloading Tuner from mcdropconnect/tuning/mcdropconnect/tuner0.json
456/456 [==============================] - 1s 1ms/step
test subject 3
Reloading Tuner from mcdropout/tuning/mcdropout/tuner0.json
Reloading Tuner from mcdropconnect/tuning/mcdropconnect/tuner0.json
456/456 [==============================] - 0s 900us/step
test subject 4
Reloading Tuner from mcdropout/tuning/mcdropout/tuner0.json
Reloading Tuner from mcdropconnect/tuning/mcdropconnect/tuner0.json
456/456 [==============================] - 0s 987us/step
test subject 5
Reloading Tuner from mcdropout/tuning/mcdropout/tuner0.json
Reloading Tuner from mcdropconnect/tuning/mcdropconnect/tuner0.json
45

/home/pmanivannan/miniconda3/envs/tf/lib/python3.9/site-packages/tensorflow/python/util/dispatch.py:1260: UserWarning: `tf.keras.backend.random_binomial` is deprecated, and will be removed in a future version.Please use `tf.keras.backend.random_bernoulli` instead.
  return dispatch_target(*args, **kwargs)


ValueError: Received incompatible tensor with shape (100,) when attempting to restore variable with shape (10,) and name dense_20/bias:0.

In [11]:
# Number of seconds to predict for all 576 samples
print(f'size of X_test: {loaded_inputs[0].shape[0]}')
for method, lsts in prediction_times.items():
    print(f'\n{method}')
    print(f'{np.mean(lsts) / 576} +/- {np.std(lsts) / 576}\n')

size of X_test: 576

standard_dropout
0.0006544805695245295 +/- 0.0007932512518761263


flipout
0.0613896119189851 +/- 0.002417636391109394


duq
0.0004499332036501096 +/- 0.00012877452541305932


ensemble_dropout
0.003791232765824706 +/- 3.222919542315011e-05


standard_dropconnect
0.0003554271879019561 +/- 5.005672764175985e-06


mcdropconnect
0.057862511111630335 +/- 0.0016170011991457124


mcdropout
0.058955910811085754 +/- 0.0009694172406823592



size of X_test: 576

standard_dropout
0.0006544805695245295 +/- 0.0007932512518761263


flipout
0.0613896119189851 +/- 0.002417636391109394


duq
0.0004499332036501096 +/- 0.00012877452541305932


ensemble_dropout
0.003791232765824706 +/- 3.222919542315011e-05


standard_dropconnect
0.0003554271879019561 +/- 5.005672764175985e-06


mcdropconnect
0.057862511111630335 +/- 0.0016170011991457124


mcdropout
0.058955910811085754 +/- 0.0009694172406823592